In [ ]:
%matplotlib notebook
from src.main.mando_preprocessing import *
import numpy as np
import matplotlib.pyplot as plt
import os
import json

In [ ]:
# Path to the raw data file

In [ ]:
def load_data(path):
    # Find the weight file in the directory
    files = [f for f in os.listdir(path) if f.startswith('weights_') and f.endswith('.txt')]
    if not files:
        raise ValueError(f"No weight files found in directory: {path}")
    # We assume there's only one file matching the pattern, hence we take the first one
    file_path = os.path.join(path, files[0])
    # Load the data using pandas
    weights = pd.read_csv(file_path, header=None, names=['weight'])
#     weights = weights[45:] # for 7,10,14,15,16 ->50,20,23,25,45
    return weights['weight']

In [ ]:
def moving_average(data, window_size):
    """Calculate the moving average of the given data using a window of specified size."""
    cumsum_vec = np.cumsum(np.insert(data, 0, 0)) 
    return (cumsum_vec[window_size:] - cumsum_vec[:-window_size]) / window_size

In [ ]:
def create_decreasing_curve(data):
    """Ensure that the data only decreases or stays constant over time to represent only food consumption."""
    decreased_data = np.copy(data)
    for i in range(1, len(decreased_data)):
        if decreased_data[i] > decreased_data[i - 1]:
            decreased_data[i] = decreased_data[i - 1]
    return decreased_data

In [ ]:
def plot(raw_data, decreasing_data):
    """
    Plot the raw, smoothed, and decreasing step-like weight data in separate subplots sharing the x-axis.

    Parameters:
    - raw_data (np.ndarray): The raw weight data array.
    - smoothed_data (np.ndarray): The smoothed weight data array.
    - decreasing_data (np.ndarray): The decreasing step-like weight data array.
    """
    fig, axs = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
    
    # Raw data plot
    axs[0].plot(raw_data, label='Raw Data', alpha=0.5, color='green')
    axs[0].set_title('Raw Weight Data')
    axs[0].set_ylabel('Weight (grams)')
    axs[0].legend()

    # # Smoothed data plot
    # axs[1].plot(smoothed_data, label='Smoothed Data', color='orange')
    # axs[1].set_title('Smoothed Weight Data')
    # axs[1].set_ylabel('Weight (grams)')
    # axs[1].legend()

    # Decreasing step-like data plot
    axs[1].plot(decreasing_data, label='Decreasing Step-like Data', color='red')
    axs[1].set_title('Decreasing Step-like Weight Data')
    axs[1].set_xlabel('Time (arbitrary units)')
    axs[1].set_ylabel('Weight (grams)')
    axs[1].legend()
    
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()


In [ ]:
def calculate_bite_sizes(decreasing_data, threshold=4):
    """
    Calculate the number of bites and the weight of each bite from the decreasing step-like weight data,
    considering only those bites where the weight decrease is greater than a specified threshold.

    Parameters:
    - decreasing_data (np.ndarray): The decreasing step-like weight data array.
    - threshold (int): The minimum weight decrease to qualify as a bite.

    Returns:
    - int: The number of bites.
    - list: The list of weights for each qualified bite.
    """
    bite_sizes = []
    for i in range(1, len(decreasing_data)):
        if decreasing_data[i] < decreasing_data[i - 1]:
            bite_size = decreasing_data[i - 1] - decreasing_data[i]
            if bite_size >= threshold:
                bite_sizes.append(bite_size)
    
    number_of_bites = len(bite_sizes)
    return number_of_bites, bite_sizes

# You would then call this function as before in the main function.


In [ ]:
from scipy.signal import find_peaks, savgol_filter

def calculate_bite_sizes_advanced(decreasing_data, threshold=4, window_length=21, polyorder=3):
    """
    Advanced calculation of the number of bites and the weight of each bite using signal processing
    techniques to detect the local minima in the smoothed signal.

    Parameters:
    - decreasing_data (np.ndarray): The decreasing step-like weight data array.
    - threshold (int): The minimum weight decrease to qualify as a bite.
    - window_length (int): The length of the filter window (number of coefficients). Must be a positive odd integer.
    - polyorder (int): The order of the polynomial used to fit the samples. Must be less than `window_length`.

    Returns:
    - int: The number of bites.
    - list: The list of weights for each qualified bite.
    """
    # Apply a Savitzky-Golay filter to the data to smooth it while preserving peaks
    smoothed_data = savgol_filter(decreasing_data, window_length=window_length, polyorder=polyorder, mode='interp')

    # Compute the first derivative of the smoothed data
    derivative = np.diff(smoothed_data, n=1)
    
    # Find peaks (local minima) in the negative derivative (which correspond to bites in the original data)
    peaks, _ = find_peaks(-derivative, height=-threshold)

    # Calculate bite sizes based on the peaks detected
    bite_sizes = np.diff(peaks, prepend=0)
    
    # Filter out the consecutive bites that are too close to each other, if necessary
    # This step is optional and can be customized based on domain knowledge
    
    number_of_bites = len(bite_sizes)
    return number_of_bites, bite_sizes.tolist()

# You would call this function in your main function after processing the data.


In [ ]:
def calculate_bite_statistics(decreasing_data, threshold=4, min_bite_interval=10):
    """
    Calculate bite statistics by considering bite size, interval, and typical eating patterns.

    Parameters:
    - decreasing_data (np.ndarray): The decreasing step-like weight data array.
    - threshold (int): The minimum weight decrease to qualify as a bite.
    - min_bite_interval (int): The minimum number of seconds expected between bites.

    Returns:
    - int: The number of qualified bites.
    - list: The list of weights for each qualified bite.
    - float: The average interval between qualified bites.
    """
    bite_sizes = []
    bite_intervals = []
    last_bite_time = 0

    for i in range(1, len(decreasing_data)):
        if decreasing_data[i] < decreasing_data[i - 1]:
            bite_size = decreasing_data[i - 1] - decreasing_data[i]
            if bite_size >= threshold:
                bite_time = i
                bite_interval = bite_time - last_bite_time

                if bite_interval >= min_bite_interval or last_bite_time == 0:
                    bite_sizes.append(bite_size)
                    bite_intervals.append(bite_interval)
                    last_bite_time = bite_time

    number_of_bites = len(bite_sizes)
    average_bite_interval = np.mean(bite_intervals) if bite_intervals else 0
    return number_of_bites, bite_sizes, average_bite_interval

# Call this function in your main function after creating the decreasing_weights curve


In [ ]:
def dif(og, c):
    print(f"Original: {og}\nCalculated: {c}")
    print(f"Difference original - calculated = {og-c}")
    print(f"Percentage: {(float)(og-c)/og}")

In [ ]:
with open('../config/mando_config.json') as config_file:
    config = json.load(config_file)
bites = config['ground_truth']['num_of_bites']
path = '../data/raw/19'
# Load the raw data
weights = load_data(path).to_numpy()
window_size = 5
# smoothed_weights = moving_average(weights, window_size) # loses step-like form
decreasing_data = create_decreasing_curve(weights)
trimmed_raw_weights = weights[window_size - 1:]
number_of_bites, bite_sizes = calculate_bite_sizes(decreasing_data,2)

print(f"Number of bites simple: {number_of_bites}")
print(f"Weight of each bite simple: {bite_sizes}")
# plot(trimmed_raw_weights, decreasing_data

adv_number_of_bites, adv_bite_sizes, bite_interval = calculate_bite_statistics(decreasing_data, 3, 2)

# dif(bites[int(path[-1])-1],adv_number_of_bites)
dif(bites[19-1],adv_number_of_bites)

plot(weights, decreasing_data)